In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pitchfork-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F655%2F1252%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240223%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240223T110452Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D000497dc1cd8dcde9b15b8ed2efca05d8a4cfac474aee4d800e2da83b3c27287ae9c3bda0bcae869e25a1c82fb7587431ca645d7d3ceae0a2353bcfdb0410e2eb016f4b62cf7bfbb6f3e2430b78fc6d5f366dac68d275d0c2a7d18b068cfb0498f1e1a851ec6ecd0afd7045db70524388a4ed10e4107998daff409354051f761d33d5c842bf9f7c153cee6360b82b31f1cc116580c18c54e708c203661a2b7351d50c576c91a4a750374263e6411bff4c1423612e66ce82798c8e4f91ba573a5073c21e76117c2905588c0f47f1161f0d9b84fb5a70e94d2961bb6cf2610f1bf824d540d23a01f9fda8f85dae50bbf707c10576bd763e644bf0cd600e002d8b3,chess:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2321%2F3919%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240223%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240223T110452Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1d296d33a2b9a18ea76bb1cadce3a7f26859c59c3e872e103ac7cb120dc595cd66b3b77a2a3442da0c69cfbb9d2132a4ed6e71a2247ada01cf8616d76fc532ea6e50c47514411fbccc37949756e39606c833d74b7a0faca203ac95d794654d4dcd936ffb36c48623e43d6404540406dfd003ace60e93e8ee41542f052a763529a96996f09cbf44b5703e24616a4358e9dc681d41664cb07f944cde227bb77f8db132eb0d735160e9cb625a9ac73bd89e8b039f0259b94adbbdaf735cf25da853975677118eed2f8c02be4af583f6b230bdaf6b2d79591ebbca0facf90396126c7ac1b411e2ef0a96d7b963d1f7b7d10fefb3ac18d0ab6dbbbc553b40fa1c5de8,kepler-exoplanet-search-results:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2894%2F4877%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240223%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240223T110452Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3f2e959ac9226d2997e8aefb9c1fbede40a829797332fc9a041bb2105c985358662d3f212c89e297b28ed0112ae982f4ccddae848c05072ed053e24d0ed575d0f39fe4c799cfc31fd9d620f7a3921623a2aadbdd000cf883a52ab516399440e5107f87aa2a7875a6c83d7377b633934a20dea9f3b927ae7d5247a662345a54a3144eab195a302cc2c99938a693c33f917da06b3b79d7353611873a419878189e978f6a97c6179ce43776e91e91ab66a543de7190192bf3f2f5b5e27f8ae284201461c1bf2ddb5199a7318b7faa0a8e56705bb2c10b0eb9bff264994490a948c6d17f10c95598e30999f5590df227bcd5ef3c6dcab5ff6245b386730b4928eced,things-on-reddit:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3491%2F5624%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240223%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240223T110452Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3175f5a633e493441d945a1da38054edd02c385550f3b764b47328772bf8562333ee77a0ace163da4e4d70276604d1271f2d9ae88d1fbe6b54521ce8d8adbfbdd8885297cc21e68152f4eb09843eef77c3dfeefd4927b0f91d0230ef170cfeb75b28cc2a91e25ee3a9643870364ac5f313014b24a6456abb652a1ad6be7103c0cfbd8b9b02550cee0dfa89b903db1281a0aa16d46122b2246677d700f4ddc476bd803706764cd28a5a4fcca4d6bcbfe1111e3b12ed59c3d3b7f5c881c846f70b0eadd39ac08b705d435f7371b235fcc5ec801854d5a3261a97da409a41e8f2736a054fd52f7fc60e68324d3670fe00fd418ceb89c8effd68371c876d17faa89e,wine-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1442%2F8172%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240223%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240223T110452Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1a26483cf9518c346b5a4f010c3d8a80adb85e9b7f475023ce95d9afb538c4ba35e2448a98dc778159c8557462751986be35b3505e936f8683d10acd250f350a3d9363d5c97974d48381ce85f58da18facecbcce3b46eaf68d5c005cd00b2377da1b6391c68b9917f9cbf448a84acac7244134c553a7ffc70c869a31d0097ed71c7088e031ca93becf22aa165115e4125973a45e2a15a3e2e5219ae3ab58d574b2cd51f1e8fe59791a7656d32cc92d23e8b643f82168bcb22fe921131d82f84654203bb7711112c749aa938e226b5faf082393cebe756993cf00cf12859d2325847d7bbda2525b6373b70543c752ab8e04e5cabeff360891ee817186a8c4452c,ramen-ratings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9366%2F13206%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240223%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240223T110452Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D490358529f14b34bed456703f5f0490d8a70a721c0afcae5ae8a42d90e1e35bbb3152b2666e76478b5eaecf520a0415385bbfcf12777c7088adcf6b3dcdb845548ae94efd529c9eca6d40cc5c11a94fdc23f91e8b013d6eb4e0b755c4097534398d6c05ae1f2912eb333ba5480b6d4c0d4648d99529857df832ab3ae89b34d39e2297e6b52a1c70e23f179a8b5524d648f18e851bb9a1b0e03774cc463f29895d512a4fbe52cecb78f28baa9525220ec837faa89c767c09c1b2f9b20debc5524a53a9723654759272a8d6373e95734d5826dd2370d606cb4516f6d4e639c0c864c9f90f2e9a87e0b40cc2a4d174b0493f1c52ec4a81f3b4ac8befd02681de765,powerlifting-database:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F179555%2F403916%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240223%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240223T110452Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2b0e9f6653a8ca385921c7bbe2dff3998aabadbefd42323d91a95122164edae33d1e593789f981d967596d2419e5acebf3e44d2b08bd0033254d8db5807d2ec57db8b2cb76c89b566d65a3949d0dbbffff41d5224e958d3e7f9ebdb7e7b3f278bdb8c74313156189feb00568458870b3b95102324b4ccc71ca5806b43b6260f15e5b3f9d8e57060d768e416739b548f3a658e32761f2ac6f4bef0af4466d68b898cd77d38c184fedf2aa9aa5c43448a8e2ab9e164a7876600986fa5fa5cb666c5f98f9ea0099607a6ddd500aaeee0b5c8fa0cb29208f8c1db1234b6edc06fad9f74c25a3f0425b89481341de78a7ea10161f3058339e9efae152ffd9f4699d46,youtube-new:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4549%2F466349%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240223%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240223T110452Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da4f9e84b51bb4664371b5b6a6d263f5cfdf206f12a02de13690854b22a8f22ad671e4c12140c33e69e63baf8a8b6c72def995bfb9faae3b4fa33fba9f95fd6108ee09751866583494aecd9cb5db50d9a23a9bee5211dfffd0b79fd6074c7dbdc20abcfa2372b04d3d0215ca56fd5fce48dfd2e946f7a472613047c9a1e4fe974706e161f2091ec49a47cb6b3237ab6e781eb2e3f9569cc03f1d16d9ced5bfabf5ae02e5df17dfa5713a615d18c2670a19ad29957346d6eaa81df6c8fdb86eef751459b4e2efc0f49ebd853c387cc002e6a2174ac13b8cbcb9f8af7f7fab911789a3f71464c1c5353da35f3782e51c57d7a2405b70e52afd1f9a45a5fec6cac76,188-million-us-wildfires:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2478%2F1151655%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240223%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240223T110452Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D19cd418b05c07e7bba1d5b7f999b4965ebc68f6e4447c789e389e5b85961b42842c9ee02bbe05d4df18717182309299fc2542719bc46be4106c2c74244ee4ce10435ce1cfd3b842c3e16eaf38d90cebe828604ddbe54c6b069b405e722619d38253a0e5a84290fe92e4249d6ddb6d6b0616b59d8da43e6f17ed3012dc21582bc944bb23faa7aeb031cc397d7ecb5b65dc110e3e2a58897520ab74f0b15eb058ba648c3f1fb4abe7e090234aa63d4849f10c38999fb9faf9fdf953f936587af68d2c06ab1a6a239564bf54230f4ea1ee8992bca31b68a25459c403984a7180e0411d7bc6f2f75c63b22154076a1cb108d64f09cc8728f5ac87235f04d41158042,publicassistance:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F10128%2F5438389%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240223%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240223T110452Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D01f9fb4831d9af75eaea41cf1ff496ce49a84b6b6faa0dc7d55b014f4924edd3fad430c11e6e1c7a25cbd0ee3db42ad0b1e494c60dc31ca4b4d36cd25e8e388f4c9eb54f21850d14d7a4f9bd8c8c3fb9cf453f129221a8f0e89c19a39c12220807c3860dc21ea9cc37559d5eed48c0558fe65b2c5d02515fa13a7cf1d4641a53a4d7444fde06687bacb5f2889d1664fb551cf4e1cdaf5114bfb4aa4180b3bb37445d60623d1217750a8dc0e044cab309510779e07826d1d8cddf96064c15ca781707bded5534c74530856f04f05275723aa5257f5ece8caea7b8571736f206b66fdbe847fe61da7d91b8cebfc1fd753f03e9ebd823f5a905c1cc0d3a0439d089'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Pandas](https://www.kaggle.com/learn/pandas) course.  You can reference the tutorial at [this link](https://www.kaggle.com/residentmario/creating-reading-and-writing).**

---


# Introduction

The first step in most data analytics projects is reading the data file. In this exercise, you'll create Series and DataFrame objects, both by hand and by reading data files.

Run the code cell below to load libraries you will need (including code to check your answers).

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 5)
from learntools.core import binder; binder.bind(globals())
from learntools.pandas.creating_reading_and_writing import *
print("Setup complete.")

Setup complete.


# Exercises

## 1.

In the cell below, create a DataFrame `fruits` that looks like this:

![](https://storage.googleapis.com/kaggle-media/learn/images/Ax3pp2A.png)

In [ ]:
# Your code goes here. Create a dataframe matching the above diagram and assign it to the variable fruits.
fruits = pd.DataFrame({'Apples': 30, 'Bananas': 21}, index=[0])

# Check your answer
q1.check()
fruits

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,Apples,Bananas
0,30,21


In [ ]:
#q1.hint()
#q1.solution()

## 2.

Create a dataframe `fruit_sales` that matches the diagram below:

![](https://storage.googleapis.com/kaggle-media/learn/images/CHPn7ZF.png)

In [ ]:
# Your code goes here. Create a dataframe matching the above diagram and assign it to the variable fruit_sales.
fruit_sales = pd.DataFrame({'Apples': [35, 41], 'Bananas': [21, 34]}, index=['2017 Sales', '2018 Sales'])

# Check your answer
q2.check()
fruit_sales

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,Apples,Bananas
2017 Sales,35,21
2018 Sales,41,34


In [ ]:
#q2.hint()
#q2.solution()

## 3.

Create a variable `ingredients` with a Series that looks like:

```
Flour     4 cups
Milk       1 cup
Eggs     2 large
Spam       1 can
Name: Dinner, dtype: object
```

In [ ]:
ingredients = pd.Series(['4 cups', '1 cup', '2 large', '1 can'], index=['Flour', 'Milk', 'Eggs', 'Spam'], name='Dinner')

# Check your answer
q3.check()
ingredients

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

Flour     4 cups
Milk       1 cup
Eggs     2 large
Spam       1 can
Name: Dinner, dtype: object

In [ ]:
#q3.hint()
#q3.solution()

## 4.

Read the following csv dataset of wine reviews into a DataFrame called `reviews`:

![](https://storage.googleapis.com/kaggle-media/learn/images/74RCZtU.png)

The filepath to the csv file is `../input/wine-reviews/winemag-data_first150k.csv`. The first few lines look like:

```
,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,"This tremendous 100% varietal wine[...]",Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and[...]",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,,Tinta de Toro,Bodega Carmen Rodríguez
```

In [ ]:
reviews = pd.read_csv("../input/wine-reviews/winemag-data_first150k.csv", index_col=0)

# Check your answer
q4.check()
reviews

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
...,...,...,...,...,...,...,...,...,...,...
150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut Rosé,90,52.0,Champagne,Champagne,NaN,Champagne Blend,Gosset
150929,Italy,More Pinot Grigios should taste like this. A r...,NaN,90,15.0,Northeastern Italy,Alto Adige,NaN,Pinot Grigio,Alois Lageder


In [ ]:
#q4.hint()
#q4.solution()

## 5.

Run the cell below to create and display a DataFrame called `animals`:

In [ ]:
animals = pd.DataFrame({'Cows': [12, 20], 'Goats': [22, 19]}, index=['Year 1', 'Year 2'])
animals

,Cows,Goats
Year 1,12,22
Year 2,20,19


In the cell below, write code to save this DataFrame to disk as a csv file with the name `cows_and_goats.csv`.

In [ ]:
# Your code goes here

# Check your answer
q5.check()

In [ ]:
#q5.hint()
#q5.solution()

# Keep going

Move on to learn about **[indexing, selecting and assigning](https://www.kaggle.com/residentmario/indexing-selecting-assigning)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/pandas/discussion) to chat with other learners.*